# Benchmark sglang Server with inference-perf

### Local sglang Setup using docker

Run sglang Server as a docker container with the model HuggingFace `HuggingFaceTB/SmolLM2-135M-Instruct`

In [1]:
!docker-compose up -d

Creating network "sglang_default" with the default driver
Creating sglang_sglang-server_1 ... 
Creating sglang_prometheus_1    ... 
ting sglang_sglang-server_1 ... done

**Note**: Configure [Sglang engine arguments](https://docs.sglang.ai/advanced_features/server_arguments.html) like `--context-length` and  `--mem-fraction-static` according to local compute capacity

### Benchmark with inference_perf

Create a configuration file for the test using `shareGPT` data and run the constant rate test for `30s`. You can also use any of the other data generators like `random`, `shared-prefix` or `synthetic` with their own configuration using the corresponding `config-*.yml` file instead.

In [2]:
!cat config.yml

data:
  type: shareGPT
load:
  type: constant
  stages:
  - rate: 1
    duration: 30
api: 
  type: chat
server:
  type: sglang
  model_name: HuggingFaceTB/SmolLM2-135M-Instruct
  base_url: http://0.0.0.0:8000
metrics:
  type: prometheus
  prometheus:
    url: http://localhost:9090
    scrape_interval: 15

In [3]:
!inference-perf --config_file config.yml

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
2025-08-25 07:47:26,589 - inference_perf.config - INFO - Using configuration from: config.yml
2025-08-25 07:47:26,592 - inference_perf.config - INFO - Benchmarking with the following config:

api:
  type: chat
  streaming: false
  headers: null
data:
  type: shareGPT
  path: null
  input_distribution: null
  output_distribution: null
  shared_prefix: null
load:
  type: constant
  interval: 1.0
  stages:
  - rate: 1
    duration: 30
  num_workers: 20
  worker_max_concurrency: 100
  worker_max_tcp_connections: 2500
metrics:
  type: prometheus
  prometheus:
    url: http://localhost:9090
    scrape_interval: 15
report:
  request_lifecycle:
    summary: true
    per_stage: true
    per_request: false
  prometheus:
    summary: true
    per_stage: false
storage:
  local_storage:
    path: reports-20250825-074726
    report_file_prefix

View benchmark metrics in the reports folder created.

### Cleanup

Delete vLLM Server docker processes

In [4]:
!docker-compose down

Stopping sglang_sglang-server_1 ... 
Stopping sglang_prometheus_1    ... 
Removing sglang_sglang-server_1 ... mdone
Removing sglang_prometheus_1    ... 
Removing network sglang_default done
